In [6]:
!pip install nlp-pie
#!pip install pie-extended

  Using cached nlp_pie-0.3.8-py2.py3-none-any.whl (84 kB)
  Using cached PyYAML-5.1b3.tar.gz (273 kB)
  Preparing metadata (setup.py) ... done
  Using cached nlp_pie-0.3.7-py2.py3-none-any.whl (84 kB)
  Using cached scikit-learn-0.22.2.post1.tar.gz (6.9 MB)
  Preparing metadata (setup.py) ... done
  Using cached nlp_pie-0.3.6-py2.py3-none-any.whl (84 kB)
  Using cached nlp_pie-0.3.5-py2.py3-none-any.whl (84 kB)
  Using cached nlp_pie-0.3.4-py2.py3-none-any.whl (84 kB)
  Using cached nlp_pie-0.3.3-py2.py3-none-any.whl (83 kB)
  Using cached nlp_pie-0.3.2-py2.py3-none-any.whl (82 kB)
  Using cached tqdm-4.23.3-py2.py3-none-any.whl (42 kB)
  Using cached gensim-3.4.0.tar.gz (22.2 MB)
  Preparing metadata (setup.py) ... done
  Using cached lxml-4.2.1.tar.gz (4.3 MB)
  Preparing metadata (setup.py) ... done
  Using cached nlp_pie-0.3.1-py2.py3-none-any.whl (82 kB)
  Using cached nlp_pie-0.3.0-py2.py3-none-any.whl (82 kB)
  Using cached nlp_pie-0.2.8-py2.py3-none-any.whl (79 kB)
  Using cach

In [3]:
from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download
from pie_extended.models. dum.imports import get_iterator_and_processor

In [4]:
do_download = True
if do_download:
    for dl in download("dum"):
        x = 1

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [5]:
iterator, processor = get_iterator_and_processor()
tagger = get_tagger('dum', batch_size=2048, device="cpu", model_path=None)

In [6]:
from glob import glob #alle bestandspaden terugvinden die overeenkomen met specifiek patroon
import os #aanmaken van mappen, verwijderen van mappen,...
from itertools import combinations #reeks verzameling cijfers/letters, combinaties in lexicografische volgorde
import grapheme #voor stringmanipulatie & rekenfucties -> stel je hebt x = 'n̄', dan len(x) = 2 (want 2 bytes), grapheme.length(x) is dan wel 1
from collatex import * #alle functies van collatex importeren, * geeft weer dat het om alle functies gaat
from tqdm import tqdm #progress meters toevoegen
import numpy as np #numpy dient om met lijsten/arrays te werken 
import pandas as pd #gebruik maken van de functies van Panda, zoals pd.DataFrame
import seaborn as sb #statistical data visualization
import matplotlib.pyplot as plt #creating static/animated/interactive visualizations

from lxml import etree
from re import sub #re — Regular expression operations #
import xml.etree.ElementTree as ET
import string

In [7]:
sigles = [os.path.basename(fn).replace('xml_', '').replace('.xml', '') for fn in glob('../collateconstraints/data/xml_martijn_lemmatizer/*.xml')] 
sigles = sorted(sigles)
sigles

['A',
 'Ant',
 'B',
 'Br',
 'C',
 'D',
 'D2',
 'E',
 'F',
 'G',
 'Ge',
 'K',
 'L',
 'O',
 'W',
 'Y',
 'Z']

In [8]:
def get_gap_lines(tree):
    gap_lines = []
    for text in tree.iterfind('.//'+"{"+ NSMAP["MVN"]+ "}"+'text'):
        if 'n' in text.attrib:
            for line in text.iterfind('.//'+"{"+ NSMAP["MVN"]+ "}"+'l'):
                if(line.find('.//'+"{"+ NSMAP["MVN"]+ "}"+'gap')) is not None:
                    gap_lines.append(line.attrib['n'])
                 #   text.attrib['n'] + '-' + 
    return gap_lines

In [9]:
NSMAP = {'MVN': 'http://www.tei-c.org/ns/1.0'} 
removes = ('teiHeader', 'fw', 'supplied', 'abbr') 
removes_expan_false = ('teiHeader', 'fw', 'supplied', 'ex', 'expan')
chars = {'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l','m', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'}       

In [10]:
def extract_lines(xml_file, expan = True, 
                  punct = True, lower = True,
                  sep_abbr = True): 
    
    lines = {}
    damaged_lines = {}
    tree = etree.parse(xml_file) 
     
    if expan:
        etree.strip_elements(tree, ("{"+ NSMAP["MVN"]+ "}" + s for s in removes), with_tail=False) 
    else: 
        etree.strip_elements(tree, ("{"+ NSMAP["MVN"]+ "}" + s for s in removes_expan_false), with_tail=False)
  
    context = etree.iterwalk(tree, events=("start", "end")) 
    text = u"" 
    k = ''     
    
    for action, node in context:   
        tag_only = node.tag.replace("{http://www.tei-c.org/ns/1.0}","") 

        if 'n' in node.attrib and tag_only == 'text': 
            title = node.attrib['n'] 
    
        if 'n' in node.attrib and tag_only == "l":   
            k = node.attrib['n']
            #print (k)
           # title + '-' + 
            
        if action == 'start' and tag_only == 'text': 
            continue
            
        elif action == 'start' and tag_only == 'lg':
            continue 
  
        elif action == 'start' and tag_only == 'lb':
            continue

        elif tag_only in ("group","text","MVN","body","cb","p"):
            continue
            
         
        elif action == 'start':
                    
            if tag_only == 'g':
                if sep_abbr: 
                    if node.attrib['ref'] == '#bar': # ā, ē, ī, ō, ū, n̄ etc.
                        text += u'\u005f' #low line _

                    elif node.attrib['ref'] == '#apomod': # ʼ
                        text += u'\u02bc'

                    elif node.attrib['ref'] == '#usmod': # ꝰ
                        text += u'\ua770' 

                    elif node.attrib['ref'] == '#condes': # ꝯ
                        text += u'\ua76f'

                    elif node.attrib['ref'] == '#para': # ¶
                        text += u'\xb6'

                    elif node.attrib['ref'] == '#etfin': # ꝫ
                        text += u'\ua76b'

                    elif node.attrib['ref'] == '#pbardes': # ꝑ
                        text += '\ua751'

                    elif node.attrib['ref'] == '#pbardes': # ꝕ
                        text += u'\ua755'

                    elif node.attrib['ref'] == '#pflour': # ꝓ
                        text += u'\ua753'

                    else:
                        text += str(node.attrib['ref']) # get the actual ref if there still are any left
                    
                else:
                    if node.attrib['ref'] == '#bar': # ā, ē, ī, ō, ū, n̄ etc.
                        text += u'\u0304'

                    elif node.attrib['ref'] == '#apomod': # ʼ
                        text += u'\u02bc'

                    elif node.attrib['ref'] == '#usmod': # ꝰ
                        text += u'\ua770'

                    elif node.attrib['ref'] == '#condes': # ꝯ
                        text += u'\ua76f'

                    elif node.attrib['ref'] == '#para': # ¶
                        text += u'\xb6'

                    elif node.attrib['ref'] == '#etfin': # ꝫ
                        text += u'\ua76b'

                    elif node.attrib['ref'] == '#pbardes': # ꝑ
                        text += u'\ua751'

                    elif node.attrib['ref'] == '#pbardes': # ꝕ
                        text += u'\ua755'

                    elif node.attrib['ref'] == '#pflour': # ꝓ
                        text += u'\ua753'

                    else:
                        node.attrib['ref']
                        text += str(node.attrib['ref']) 
            superscript_dict = {'a':'ᵃ', 'b':'ᵇ', 'c':'ᶜ', 'd':'ᵈ', 'e':'ᵉ', 'f':'ᶠ',
                               'g':'ᵍ', 'h':'ʰ', 'i':'ᶦ', 'j':'ʲ', 'k':'ᵏ', 'l':'ˡ', 
                                'm':'ᵐ', 'n':'ⁿ', 'o':'ᵒ', 'p':'ᵖ', 'r':'ʳ', 's':'ˢ', 
                                't':'ᵗ', 'u':'ᵘ', 'v':'ᵛ', 'w':'ʷ', 'x':'ˣ', 'y': 'ʸ', 'z': 'ᶻ'}

            if tag_only == 'hi' and 'rend' in node.attrib and node.attrib['rend'] == 'superscript': #rend(ition) supplies information about the appearance of an element
                if node.text in superscript_dict:
                    text += str(superscript_dict[node.text]).strip()

            elif tag_only == 'pc':
                text += str(node.text).strip()

            elif tag_only == 'num':
                if node.text:
                    text += str('.'+node.text+'.').strip()

            elif (node.text):
                text += node.text        
        elif action == 'end':
            
            if (node.tail and node.tail not in "\t"): 
                text += str(node.tail)
                
        if tag_only == 'lb':
            if k:
                text = sub(r'\n', '', text) 
                if not punct:
                    text = text.translate(str.maketrans('', '', string.punctuation)) 
                if lower: 
                    text = text.lower()
                   
                    #text = text[::-1]
                  
                lines[k] = text #the value of the keys k is the text 
                text = ''  
                   
   
    if text:
        lines[k] = (text)
   
    num_orig_lines = len(lines)
    print(num_orig_lines)
    # remove lines with gaps:
    gap_lines = get_gap_lines(tree)
    #print(gap_lines)
    #lines = {k:v for k, v in lines.items()}
    for k, v in lines.items():
        if k in gap_lines:
            damaged_lines[k] = ('True')
        else:
            damaged_lines[k] = ('False')
    #lines = {k:v for k, v in lines.items() if v.strip()} #if a line with a gap is removed, remove empty key, strip() removes spaces #The items() method returns a key-value pair
    
    return lines, damaged_lines
   

d = extract_lines(f'../collateconstraints/data/xml_martijn_lemmatizer/xml_{sigles[0]}.xml', expan = True, punct = False, lower = True)
#print(d)

1766


In [11]:
#for sigle in sigles: 
  #  d = extract_lines(f'../collateconstraints/data/xml_martijn/xml_{sigle}.xml', expan = True, punct = False, lower = True)
    #print(d)

mss = {} #dictionary maken van manuscripten

for sigle in tqdm(sigles): #for... in = loop, toont progress meter 
    mss[sigle] = extract_lines(f'../collateconstraints/data/xml_martijn_lemmatizer/xml_{sigle}.xml',
                               expan = True, punct = False, lower = True,
                               sep_abbr = False)

#print(mss['A'])

  0%|                                                    | 0/17 [00:00<?, ?it/s]

1766
104


 24%|██████████▎                                 | 4/17 [00:00<00:00, 34.89it/s]

1816
606
1472
1811
247
276
1821


 59%|█████████████████████████▎                 | 10/17 [00:00<00:00, 35.45it/s]

1768
152
67


100%|███████████████████████████████████████████| 17/17 [00:00<00:00, 44.15it/s]

703
1821
507
348
534


In [12]:
lines = []
keys = []  # Create an empty list to store the keys

for sigle in sigles: 
    for key, line in mss[sigle][0].items():  # Use the items() method to iterate over both keys and values
        clean_line = []
        for token in line.split():
            token = ''.join([c for c in token.lower() if c.isalpha()])
            clean_line.append(token)
        lines.append(' '.join(clean_line))
        keys.append(key)  # Append the key to the list
# Print keys and lines together
    
#for key, line in zip(keys, lines):
  #  print(key, line)

In [13]:
import csv
for sigle in sigles:
    filename = f"data/tsv_files/{sigle}_output.tsv"
    lines = []
    keys = []  # Create an empty list to store the keys

    for key, line in mss[sigle][0].items():  # Use the items() method to iterate over both keys and values
        clean_line = []
        for token in line.split():
            token = ''.join([c for c in token.lower() if c.isalpha()])
            clean_line.append(token)
        lines.append(' '.join(clean_line))
        keys.append(key)  # Append the key to the list

    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file, delimiter='\t', quoting=csv.QUOTE_NONE, escapechar='')

            # Write the header row
        header_row = ['line_id', 'form', 'lemma', 'pos', 'corr_lemma', 'corr_pos', 'damage']  # Include 'key' in the header
        writer.writerow(header_row)

        for key, line in zip(keys, lines):  # Iterate over both keys and lines together
            damage = mss[sigle][1][key]
            for w in tagger.tag_str(line.lower(), iterator=iterator, processor=processor):
                # Write each dictionary as a row in the TSV file, including the key
                writer.writerow([key, w['form'], w['lemma'], w['pos'], w['lemma'], w['pos'], damage])

            # Add an empty line between each dictionary
            writer.writerow([])